In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore

# Load stock price data (Assumed format: Date | Ticker | Close | Volume | MarketCap | P/E | P/B)
df = pd.read_excel("brvm_stock_data.xlsx", parse_dates=["Date"])


## 📌 Step 2: Compute Factor Scores

In [ ]:
# Compute Momentum (6M, 12M returns)
df["Momentum_6M"] = df.groupby("Ticker")["Close"].pct_change(126)  # 6 months ≈ 126 trading days
df["Momentum_12M"] = df.groupby("Ticker")["Close"].pct_change(252)  # 12 months ≈ 252 trading days

# Compute Volatility (Rolling 60-day standard deviation)
df["Volatility"] = df.groupby("Ticker")["Close"].pct_change().rolling(60).std()

# Compute Liquidity (Average Daily Volume over last 3 months)
df["Liquidity"] = df.groupby("Ticker")["Volume"].rolling(63).mean().reset_index(0, drop=True)

# Standardize factors (Z-score normalization)
factors = ["Momentum_6M", "Momentum_12M", "Volatility", "Liquidity", "P/E", "P/B", "MarketCap"]
df[factors] = df[factors].apply(zscore)


## 📌 Step 3: Rank Stocks & Build Portfolio

In [ ]:
# Compute combined factor score
df["Factor_Score"] = (
    df["Momentum_6M"] * 0.25 + df["Momentum_12M"] * 0.25 +
    df["Volatility"] * -0.15 + df["Liquidity"] * 0.15 + 
    df["P/E"] * -0.10 + df["P/B"] * -0.10 + df["MarketCap"] * -0.10
)

# Rank stocks
df["Rank"] = df.groupby("Date")["Factor_Score"].rank(ascending=False)

# Select Top 20% (Long) & Bottom 20% (Short)
long_stocks = df[df["Rank"] <= df["Rank"].quantile(0.2)]
short_stocks = df[df["Rank"] >= df["Rank"].quantile(0.8)]


## 📌 Step 4: Backtest Portfolio Performance

In [ ]:
# Compute portfolio returns
df["Portfolio_Return"] = df["Factor_Score"].pct_change()
df["BRVM_Index_Return"] = df["Close"].pct_change()  # Assuming this is the BRVM Index

# Cumulative returns
df["Portfolio_CumReturn"] = (1 + df["Portfolio_Return"]).cumprod()
df["BRVM_CumReturn"] = (1 + df["BRVM_Index_Return"]).cumprod()

# Plot Performance
plt.figure(figsize=(12, 6))
plt.plot(df["Date"], df["Portfolio_CumReturn"], label="Factor-Based Portfolio", linewidth=2)
plt.plot(df["Date"], df["BRVM_CumReturn"], label="BRVM Index", linestyle="dashed")
plt.legend()
plt.title("Factor-Based Portfolio vs. BRVM Index")
plt.show()

## 📌 Step 5: Analyze Results

In [ ]:
sharpe_ratio = df["Portfolio_Return"].mean() / df["Portfolio_Return"].std()
max_drawdown = (df["Portfolio_CumReturn"] / df["Portfolio_CumReturn"].cummax() - 1).min()

print(f"📊 Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"📉 Max Drawdown: {max_drawdown:.2%}")